# Field Validators

## After Validators

In [42]:
from pydantic import BaseModel, ValidationError, field_validator


class Model(BaseModel):
    number: int

    @field_validator('number', mode='after')  
    @classmethod
    def is_even(cls, value: int) -> int:
        if value % 2 == 1:
            raise ValueError(f'{value} is not an even number')
        return value  


try:
    Model(number=1)
except ValidationError as err:
    print(err)

1 validation error for Model
number
  Value error, 1 is not an even number [type=value_error, input_value=1, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error


In [43]:
from typing import Annotated

from pydantic import AfterValidator, BaseModel, ValidationError


def is_even(value: int) -> int:
    if value % 2 == 1:
        raise ValueError(f'{value} is not an even number')
    return value  


class Model(BaseModel):
    number: Annotated[int, AfterValidator(is_even)]


try:
    Model(number=1)
except ValidationError as err:
    print(err)

1 validation error for Model
number
  Value error, 1 is not an even number [type=value_error, input_value=1, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error


## Before Validators

In [44]:
from typing import Any

from pydantic import BaseModel, ValidationError, field_validator


class Model(BaseModel):
    numbers: list[int]

    @field_validator('numbers', mode='before')
    @classmethod
    def ensure_list(cls, value: Any) -> Any:  
        if not isinstance(value, list):  
            return [value]  # it is not recommended to mutate the input data, just throw a validation error
        else:
            return value


print(Model(numbers=2))
#> numbers=[2]
try:
    Model(numbers='str')
except ValidationError as err:
    print(err)  

numbers=[2]
1 validation error for Model
numbers.0
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='str', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/int_parsing


In [45]:
from typing import Annotated, Any

from pydantic import BaseModel, BeforeValidator, ValidationError


def ensure_list(value: Any) -> Any:  
    if not isinstance(value, list):  
        return [value]  # it is not recommended to mutate the input data, just throw a validation error
    else:
        return value


class Model(BaseModel):
    numbers: Annotated[list[int], BeforeValidator(ensure_list)]


print(Model(numbers=2))
#> numbers=[2]
try:
    Model(numbers='str')
except ValidationError as err:
    print(err)  

numbers=[2]
1 validation error for Model
numbers.0
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='str', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/int_parsing


## Plain validators, like the before but Pydantic does not kick in.

In [46]:
from typing import Any

from pydantic import BaseModel, field_validator


class Model(BaseModel):
    number: int

    @field_validator('number', mode='plain')
    @classmethod
    def val_number(cls, value: Any) -> Any:
        if isinstance(value, int):
            return value * 2
        else:
            return value


print(Model(number=4))
#> number=8
print(Model(number='invalid'))  
#> number='invalid'

number=8
number='invalid'


In [47]:
from typing import Annotated, Any

from pydantic import BaseModel, PlainValidator


def val_number(value: Any) -> Any:
    if isinstance(value, int):
        return value * 2
    else:
        return value


class Model(BaseModel):
    number: Annotated[int, PlainValidator(val_number)]


print(Model(number=4))
#> number=8
print(Model(number='invalid'))  
#> number='invalid'

number=8
number='invalid'


## Wrap (Hybrid) Validators (you use optionally the Pydantic validator and do your own stuff)

In [48]:
from typing import Any

from typing import Annotated

from pydantic import BaseModel, Field, ValidationError, ValidatorFunctionWrapHandler, field_validator


class Model(BaseModel):
    my_string: Annotated[str, Field(max_length=5)]

    @field_validator('my_string', mode='wrap')
    @classmethod
    def truncate(cls, value: Any, handler: ValidatorFunctionWrapHandler) -> str:
        try:
            return handler(value)
        except ValidationError as err:
            if err.errors()[0]['type'] == 'string_too_long':
                return handler(value[:5])    # all bets are off at this point
            else:
                raise


print(Model(my_string='abcde'))
#> my_string='abcde'
print(Model(my_string='abcdef'))
#> my_string='abcde'

my_string='abcde'
my_string='abcde'


In [49]:
from typing import Any

from typing import Annotated

from pydantic import BaseModel, Field, ValidationError, ValidatorFunctionWrapHandler, WrapValidator


def truncate(value: Any, handler: ValidatorFunctionWrapHandler) -> str:
    try:
        return handler(value)
    except ValidationError as err:
        if err.errors()[0]['type'] == 'string_too_long':
            return handler(value[:5])
        else:
            raise


class Model(BaseModel):
    my_string: Annotated[str, Field(max_length=5), WrapValidator(truncate)]


print(Model(my_string='abcde'))
#> my_string='abcde'
print(Model(my_string='abcdef'))
#> my_string='abcde'

my_string='abcde'
my_string='abcde'


## Reusing an Annotated validator (impossible with the decorator)

In [50]:
from typing import Annotated

from pydantic import AfterValidator, BaseModel


def is_even(value: int) -> int:
    if value % 2 == 1:
        raise ValueError(f'{value} is not an even number')
    return value


EvenNumber = Annotated[int, AfterValidator(is_even)]


class Model1(BaseModel):
    my_number: EvenNumber


class Model2(BaseModel):
    other_number: Annotated[EvenNumber, AfterValidator(lambda v: v + 2)]


class Model3(BaseModel):
    list_of_even_numbers: list[EvenNumber]

## Multi-apply the decorator

In [51]:
from pydantic import BaseModel, field_validator


class SomeModel(BaseModel):
    f1: str
    f2: str

    # @field_validator('f1', 'f2', mode='before')
    @field_validator('*', mode='before') # similar to the above if you aim for all the fields
    @classmethod
    def capitalize(cls, value: str) -> str:
        return value.capitalize()


m = SomeModel(f1="HeLLo", f2="WORld")
assert m.model_dump() == {'f1': 'Hello', 'f2': 'World'}

# Model validators

In [52]:
# I will give the wrapper one. The rest are mostly like the above


import logging
from typing import Any

from typing_extensions import Self

from pydantic import BaseModel, ModelWrapValidatorHandler, ValidationError, model_validator


class UserModel(BaseModel):
    username: str

    @model_validator(mode='wrap')
    @classmethod
    def log_failed_validation(cls, data: Any, handler: ModelWrapValidatorHandler[Self]) -> Self:
        try:
            return handler(data)
        except ValidationError:
            logging.error('Model %s failed to validate with data %s', cls, data)
            raise

# Validation Errors

* ValueError: this is the most common exception raised inside validators.
* AssertionError: using the assert statement also works, but be aware that these statements **are skipped when Python is run with the -O optimization** flag.
* PydanticCustomError: a bit more verbose, but provides extra flexibility:

In [53]:
from pydantic_core import PydanticCustomError

from pydantic import BaseModel, ValidationError, field_validator

class Model(BaseModel):
    x: int

    @field_validator('x', mode='after')
    @classmethod
    def validate_x(cls, v: int) -> int:
        if v % 42 == 0:
            raise PydanticCustomError(
                'the_answer_error',
                '{number} is the answer!',
                {'number': v},
            )
        return v


try:
    Model(x=42 * 2)
except ValidationError as e:
    print(e)

1 validation error for Model
x
  84 is the answer! [type=the_answer_error, input_value=84, input_type=int]


In [54]:
from pydantic_core import PydanticCustomError

from pydantic import BaseModel, ValidationError, field_validator


class Model(BaseModel):
    x: int

    @field_validator('x', mode='after')
    @classmethod
    def validate_x(cls, v: int) -> int:
        if v % 42 == 0:
            raise ValueError("Number is not divisible with 84")
        return v


try:
    Model(x=42 * 2)
except ValidationError as e:
    print(e)

1 validation error for Model
x
  Value error, Number is not divisible with 84 [type=value_error, input_value=84, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error


In [55]:
from pydantic import BaseModel, ValidationInfo, field_validator


class UserModel(BaseModel):
    password: str
    password_repeat: str
    username: str

    @field_validator('password_repeat', mode='after')
    @classmethod
    def check_passwords_match(cls, value: str, info: ValidationInfo) -> str:
        print(info)
        if value != info.data['password']:
            raise ValueError('Passwords do not match')
        return value


try:
    UserModel(password='1234', password_repeat='5678', username='gojira')
except ValidationError as e:
    print(e)

ValidationInfo(config={'title': 'UserModel'}, context=None, data={'password': '1234'}, field_name='password_repeat')
1 validation error for UserModel
password_repeat
  Value error, Passwords do not match [type=value_error, input_value='5678', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error


In [56]:
from pydantic import BaseModel, ValidationInfo, field_validator


class Model(BaseModel):
    text: str

    @field_validator('text', mode='after')
    @classmethod
    def remove_stopwords(cls, v: str, info: ValidationInfo) -> str:
        if isinstance(info.context, dict):
            stopwords = info.context.get('stopwords', set())
            v = ' '.join(w for w in v.split() if w.lower() not in stopwords)
        return v


data = {'text': 'This is an example document'}
print(Model.model_validate(data))  # no context
#> text='This is an example document'
print(Model.model_validate(data, context={'stopwords': ['this', 'is', 'an']}))
#> text='example document'

text='This is an example document'
text='example document'
